# CONNECT THE BOARD

In [13]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Ready.


In [ ]:
f = open("main.py", "r")
print(f.read())

In [ ]:
%sendtofile lib/collector.py --source lib/collector.py
%sendtofile lib/peripherals.py --source lib/peripherals.py
%sendtofile lib/signal.py --source lib/signal.py

In [14]:
%sendtofile main.py --source main.py

Sent 113 lines (2823 bytes) to main.py.


# COLLECTOR METHOD

In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

from lib.requests import MicroWebCli as requests
import ujson as json

from lib.utils import connect_wifi, load_env_vars

from ulab import numpy as np
import utime as time
from lib.collector import Collector 
from lib.runner import Runner 

In [ ]:
collector = Collector(buffer_size=256, preprocess=True, ds=True)

SPI_params= {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,
}

collector.setup(spi_params=SPI_params)

# ssid = 'TP-Link_AP_4C04'
# password = '63525465'
# connect_wifi(ssid, password)
time.sleep(4)

collector.run()

time.sleep(10)

while True:
    time.sleep(1)
    bt = time.time()
    data = collector.read_output_buffer()
    et = time.time()
    print(et-bt,data)
#     toSend = {"raw_data":data}
#     print(toSend)
#     requests.JSONRequest("http://192.168.0.13:5001/collect", toSend)
#     # requests.JSONRequest("http://172.20.10.2:5001/collect", toSend)
#     del data
#     del toSend
#     # requests.GETRequest("http://172.20.10.2:5001/save")
#     requests.GETRequest("http://192.168.0.13:5001/save")

In [ ]:
collector.stop()

In [ ]:
runner = Runner('CCA', buffer_size=256)
runner.setup()

# ssid = 'VM0857951'
# password = 'cchk7mrYgrzq'
# connect_wifi(ssid, password)

time.sleep(4)
runner.run()
while True:
    time.sleep(1)
    data = runner.read_output_buffer()
    print(data)

In [ ]:
runner.stop()

# DIRECTLY READ ADC, SKIP EVERYTHING ELSE

In [ ]:
%serialconnect to --port="COM5" --baud=115200

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    **temp_spi_params
)

data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
spi.write(data)

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= 256:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=256, callback=sample_callback)

# print(adc_sample)
# print("20000 ADC readings done after %u us." %(t2-t1))
# print("Mean time for each ADC reading = %15.13f us" % ((t2-t1)/20000.0))
# print("ADC reading = %15.13f ksamples/s" % (1000/((t2-t1)/20000.0)))

In [ ]:
while True:
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

In [ ]:
requests.GETRequest("http://192.168.0.37:5001/save")

In [ ]:
%serialconnect to --port="COM5" --baud=115200

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray(DEFAULT_SPI_PARAMS['output_amp_gain'])
# data = bytearray(100)
spi.write(data)

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= 256:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=256, callback=sample_callback)

for i in range(30):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

requests.GETRequest("http://192.168.0.37:5001/save")

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 50,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray(DEFAULT_SPI_PARAMS['output_amp_gain'])
# data = bytearray(100)
spi.write(data)

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []
pot_size = 512

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

for i in range(30):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

requests.GETRequest("http://192.168.0.37:5001/save")

In [ ]:
%serialconnect to --port="COM5" --baud=115200

# TESTING GAIN

In [ ]:
%serialconnect to --port="COM5" --baud=115200

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(100)
cs = machine.Pin(5, machine.Pin.OUT)
cs.on() #active low
print("Set GPIO 5 LOW")
spi.write(data)
print("Sent gain")
cs.off() #active high
print("Set GPIO 5 HIGH")
while True:
    time.sleep(1)
    print(adc.read())

In [ ]:
# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(DEFAULT_SPI_PARAMS['output_amp_gain'])
data = bytearray(255)
spi.write(data)

In [6]:
%sendtofile main.py --source main.py

Sent 111 lines (2737 bytes) to main.py.


In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
from machine import freq

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray(DEFAULT_SPI_PARAMS['output_amp_gain'])
# data = bytearray(100)
spi.write(data)

In [ ]:
%ls lib

In [ ]:
%sendtofile lib/computation.py --source lib/computation.py

# BEFORE CHANGES TO GAIN

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 100,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray(DEFAULT_SPI_PARAMS['output_amp_gain'])
# data = bytearray(100)
spi.write(data)

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []
pot_size = 512

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

for i in range(30):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

requests.GETRequest("http://192.168.0.37:5001/save")

# after changes to gain - testing

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(100)
cs = machine.Pin(5, machine.Pin.OUT)

# have to pull GPIO 5 LOW before writing the gain

cs.on() #active low
spi.write(data)
cs.off() #active high

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 512

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

for i in range(30):
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

requests.GETRequest("http://192.168.0.37:5001/save")

In [ ]:
%serialconnect to --port="COM5" --baud=115200

# Testing live decoding

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(100)
cs = machine.Pin(5, machine.Pin.OUT)

# have to turn GPIO 5 off before writing the gain

cs.off() 
spi.write(data)
cs.on() 

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 256

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 7 HZ ###"})
time.sleep(10)
    
#send 7hz data
for i in range(4):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"7":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/7hz", toSend)
    
requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 10 HZ ###"})
time.sleep(10)

#send 10hz data
for i in range(4):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"10":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/10hz", toSend)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 12 HZ ###"})
time.sleep(5)

#send 12hz data
for i in range(4):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"12":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/12hz", toSend)

requests.GETRequest("http://192.168.0.37:5001/isCalibrated")

while True:
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/decode", toSend)

In [10]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Ready.


In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
cs = machine.Pin(5, machine.Pin.OUT)

# have to pull GPIO 5 LOW before writing the gain

cs.off() #active low
spi.write(data)
cs.on() #active high

# ssid = 'TP-Link_AP_4C04'
# password = '63525465'
# connect_wifi(ssid, password)

###############################################################################

adc_sample = []
pot_size = 512

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

for i in range(30):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    print(min(data),max(data))
    print(data)


# For board

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(100)
cs = machine.Pin(5, machine.Pin.OUT)

# have to turn GPIO 5 off before writing the gain

cs.off() 
spi.write(data)
cs.on() 

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 256

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

for i in range(30):
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

requests.GETRequest("http://192.168.0.37:5001/save")

In [5]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Ready.


In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
cs = machine.Pin(5, machine.Pin.OUT)

# have to turn GPIO 5 off before writing the gain

cs.off() 
spi.write(data)
cs.on() 

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 256

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 7 HZ ###"})
time.sleep(10)
    
#send 7hz data
for i in range(4):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"7":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/7hz", toSend)
    
requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 10 HZ ###"})
time.sleep(10)

#send 10hz data
for i in range(4):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"10":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/10hz", toSend)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 12 HZ ###"})
time.sleep(5)

#send 12hz data
for i in range(4):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"12":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/12hz", toSend)

requests.GETRequest("http://192.168.0.37:5001/isCalibrated")

while True:
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/decode", toSend)

# 64hz sampling over 4s window

In [11]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
cs = machine.Pin(5, machine.Pin.OUT)

# have to turn GPIO 5 off before writing the gain

cs.off() 
spi.write(data)
cs.on() 

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 256
sampling_rate = 64

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=sampling_rate, callback=sample_callback)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 7 HZ ###"})
time.sleep(10)

decode_period = 4
#send 7hz data
for i in range(4):
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"7":data}
    requests.JSONRequest("http://192.168.0.37:5001/7hz", toSend)
    
requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 10 HZ ###"})
time.sleep(10)

#send 10hz data
for i in range(4):
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"10":data}
    requests.JSONRequest("http://192.168.0.37:5001/10hz", toSend)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 12 HZ ###"})
time.sleep(5)

#send 12hz data
for i in range(4):
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"12":data}
    requests.JSONRequest("http://192.168.0.37:5001/12hz", toSend)

requests.GETRequest("http://192.168.0.37:5001/isCalibrated")

while True:
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"raw_data":data}
    requests.JSONRequest("http://192.168.0.37:5001/decode", toSend)

connecting to network...
network config: ('192.168.0.22', '255.255.255.0', '192.168.0.1', '194.168.4.100')
........................

*** Sending Ctrl-C



Keyboard interrupt while waiting response on Ctrl-C

